In [141]:
import pandas as pd
import numpy as np

In [142]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [143]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [144]:
meta = getDF('./meta_All_Beauty.json.gz')
ratings = pd.read_csv('./All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [145]:
category = meta['rank'].str.split(' in ', expand=True)
category = category.rename(columns={category.columns[0]:'ranking', category.columns[1]:'categories'})
category.head()

,ranking,categories
0,"2,938,573",Beauty & Personal Care (
1,"872,854",Beauty & Personal Care (
2,"956,696",Beauty & Personal Care (
3,"1,870,258",Beauty & Personal Care (
4,"67,701",Beauty & Personal Care (


In [146]:
category['ranking'] = category['ranking'].str.replace(',','')
category['categories'] = category['categories'].str.replace('amp;','')
category['categories'] = category['categories'].str.replace('\(', '')
category['ranking'] = pd.to_numeric(category['ranking'])

In [147]:
category['categories'].unique()

array(['Beauty & Personal Care ', nan, 'Grocery & Gourmet Food ',
       'Health & Household ', 'Sports & Outdoors ', 'Toys & Games ',
       'Baby ', 'Clothing, Shoes & Jewelry ', 'Home & Kitchen ',
       'Automotive ', 'Tools & Home Improvement '], dtype=object)

In [148]:
meta_use = meta[['asin', 'title', 'also_buy', 'brand', 'rank', 'also_view', 'details', 'price']]

In [149]:
meta_use.shape

(32892, 8)

In [150]:
meta_use['ranking'] = category['ranking']
meta_use['categories'] = category['categories']

<ipython-input-150-110db4146380>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_use['ranking'] = category['ranking']
<ipython-input-150-110db4146380>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_use['categories'] = category['categories']


In [151]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1500508800,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1489622400,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1488326400,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1487635200,2017-02-21


In [152]:
ratings_trainings = ratings[(ratings['DATE'] < '2018-09-01')]
ratings_testings = ratings[(ratings['DATE'] >= '2018-09-01') &  (ratings['DATE'] <= '2018-09-30')]

In [153]:
ratings.shape, ratings_trainings.shape, ratings_testings.shape

((371345, 5), (370752, 5), (590, 5))

In [154]:
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')

In [155]:
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [156]:
train_groupby = ratings_trainings.groupby(['asin']).aggregate({'overall':['mean', 'size']})['overall']
train_groupby = train_groupby.reset_index()
train_groupby = train_groupby.rename(columns={0:'size'})
train_groupby = train_groupby.sort_values(['mean'], ascending=False)

In [157]:
meta_merge = meta_use.merge(train_groupby, on='asin', how='left')
meta_merge = meta_merge.sort_values(['mean', 'size'], ascending=False)

In [158]:
a = ratings.groupby('asin').mean()['overall']#.sort_values(['mean'], ascending=False)
b = ratings.groupby('asin').size()
ratings_rank = pd.DataFrame({'mean': a, 'size': b}).reset_index().sort_values(['mean', 'size'], ascending=False)

In [159]:
ratings_rank

,asin,mean,size
1246,B000KNELAW,5.0,121
1007,B000GCPXN6,5.0,37
603,B00079PB6M,5.0,22
15166,B00MHYLOMA,5.0,22
4899,B004LTF18M,5.0,17
...,...,...,...
32552,B01HHFEA42,1.0,1
32568,B01HI6M8H6,1.0,1
32571,B01HIE3EK8,1.0,1
32582,B01HJ1K3YK,1.0,1


In [160]:
# top_10_mean_size = meta_merge[:10]

In [161]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    top_10 = training_data.groupby('asin').size().sort_values(ascending=False).index.to_list()[:10]
    recommendations = {user: top_10 for user in users}
    '''
    Your Code
    '''
    return recommendations


In [163]:
def recommender1(training_data, users=[], k=10):
  '''
  * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
  * users: [] 需要被推薦的使用者
  * k: int 每個使用者需要推薦的商品數
  * recommendations: dict
    {
        使用者一： [推薦商品一, 推薦商品二, ...],
        使用者二： [...], ...
    }
  '''
  recommendations = {}
  n=0
  for ii in users:
    user = ratings[ratings['reviewerID'] == str(ii)]
    prod = user.groupby('asin').aggregate({'overall':['mean', 'size']})['overall'].sort_values(['mean']).reset_index()
    cates = pd.DataFrame(columns={'category', 'asin'})
    cates['asin'] = prod['asin']
    if len(prod) > 0:
      for jj in range(0, len(prod)):
        if len(meta_merge[meta_merge['asin'] == prod['asin'].unique()[jj]]) > 0:
          cates['category'][jj] = meta_merge[meta_merge['asin'] == prod['asin'].unique()[jj]]['categories'].iloc[0]
          group = cates.groupby(by=['asin']).count()
      num = int(np.ceil(k/len(cates)))
      recommend_cates = []
      for kk in range(0, len(cates['category'].unique())):
        if len(cates[cates['category'] == cates['category'].unique()[kk]]) >= num:
          [recommend_cates.append(meta_merge[meta_merge['categories'] == cates['category'].unique()[kk]].iloc[gg]['asin']) for gg in range(0, num)]
        else:
          [recommend_cates.append(meta_merge[meta_merge['categories'] == cates['category'].unique()[kk]].iloc[gg]['asin']) for gg in range(0, len(cates[cates['category'] == cates['category'].unique()[kk]]))]
    
    top_10 = training_data.groupby('asin').size().sort_values(ascending=False).index.to_list()[:10]
    if len(recommend_cates) <= 10:
      [recommend_cates.append(list(ratings_rank['asin'])[gg]) for gg in range(0, 10-len(recommend_cates))]
    recommendations = {ii : recommend_cates for ii in users}
  return recommendations
# ratings_by_user = recommender(ratings_trainings, users)
# ratings_by_user


In [164]:
def recommender2(training_data, users=[], k=10):
  '''
  * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
  * users: [] 需要被推薦的使用者
  * k: int 每個使用者需要推薦的商品數
  * recommendations: dict
    {
        使用者一： [推薦商品一, 推薦商品二, ...],
        使用者二： [...], ...
    }
  '''
  recommendations = {}
  n=0
  for ii in users:
    user = ratings[ratings['reviewerID'] == str(ii)]
    prod = user.groupby('asin').aggregate({'overall':['mean', 'size']})['overall'].sort_values(['mean']).reset_index()
    cates = pd.DataFrame(columns={'category', 'asin'})
    cates['asin'] = prod['asin']
    if len(prod) > 0:
      for jj in range(0, len(prod)):
        if len(meta_merge[meta_merge['asin'] == prod['asin'].unique()[jj]]) > 0:
          cates['category'][jj] = meta_merge[meta_merge['asin'] == prod['asin'].unique()[jj]]['categories'].iloc[0]
          group = cates.groupby(by=['asin']).count()
      num = int(np.ceil(k/len(cates)))
      recommend_cates = []
      for kk in range(0, len(cates['category'].unique())):
        if len(cates[cates['category'] == cates['category'].unique()[kk]]) >= num:
          [recommend_cates.append(meta_merge[meta_merge['categories'] == cates['category'].unique()[kk]].iloc[gg]['asin']) for gg in range(0, num)]
        else:
          [recommend_cates.append(meta_merge[meta_merge['categories'] == cates['category'].unique()[kk]].iloc[gg]['asin']) for gg in range(0, len(cates[cates['category'] == cates['category'].unique()[kk]]))]
    
    top_10 = training_data.groupby('asin').size().sort_values(ascending=False).index.to_list()[:10]
    if len(recommend_cates) <= 10:
      [recommend_cates.append(top_10[gg]) for gg in range(0, 10-len(recommend_cates))]
    recommendations = {ii : recommend_cates for ii in users}
  return recommendations
# ratings_by_user = recommender(ratings_trainings, users)
# ratings_by_user


In [165]:
def recommender3(training_data, users=[], k=10):
  '''
  * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
  * users: [] 需要被推薦的使用者
  * k: int 每個使用者需要推薦的商品數
  * recommendations: dict
    {
        使用者一： [推薦商品一, 推薦商品二, ...],
        使用者二： [...], ...
    }
  '''
  training_data = training_data[(ratings['DATE'] >= '2018-06-30') &  (ratings['DATE'] <= '2018-09-01')]
  recommendations = {}
  n=0
  for ii in users:
    user = ratings[ratings['reviewerID'] == str(ii)]
    prod = user.groupby('asin').aggregate({'overall':['mean', 'size']})['overall'].sort_values(['mean']).reset_index()
    cates = pd.DataFrame(columns={'category', 'asin'})
    cates['asin'] = prod['asin']
    if len(prod) > 0:
      for jj in range(0, len(prod)):
        if len(meta_merge[meta_merge['asin'] == prod['asin'].unique()[jj]]) > 0:
          cates['category'][jj] = meta_merge[meta_merge['asin'] == prod['asin'].unique()[jj]]['categories'].iloc[0]
          group = cates.groupby(by=['asin']).count()
      num = int(np.ceil(k/len(cates)))
      recommend_cates = []
      for kk in range(0, len(cates['category'].unique())):
        if len(cates[cates['category'] == cates['category'].unique()[kk]]) >= num:
          [recommend_cates.append(meta_merge[meta_merge['categories'] == cates['category'].unique()[kk]].iloc[gg]['asin']) for gg in range(0, num)]
        else:
          [recommend_cates.append(meta_merge[meta_merge['categories'] == cates['category'].unique()[kk]].iloc[gg]['asin']) for gg in range(0, len(cates[cates['category'] == cates['category'].unique()[kk]]))]
    
    top_10 = training_data.groupby('asin').size().sort_values(ascending=False).index.to_list()[:10]
    if len(recommend_cates) <= 10:
      [recommend_cates.append(top_10[gg]) for gg in range(0, 10-len(recommend_cates))]
    recommendations = {ii : recommend_cates for ii in users}
  return recommendations
# ratings_by_user = recommender(ratings_trainings, users)
# ratings_by_user



In [166]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
    score = total / len(ratings_testings)
    return score
# evaluate(ratings_testings_by_user, ratings_by_user)

In [167]:
evaluate_result = {}
n = 0
for recommender in [recommender, recommender1, recommender2, recommender3]:
  ratings_by_user = {}
  # print(ratings_by_user)
  ratings_by_user = recommender(ratings_trainings, users)
  # print(f'{ii}\n',ratings_by_user['A142I22FIC8MZK'])
  evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
  n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])
result


<ipython-input-165-7c82c756701a>:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  training_data = training_data[(ratings['DATE'] >= '2018-06-30') &  (ratings['DATE'] <= '2018-09-01')]


,recommender_list,score
0,0,0.083051
1,1,0.000000
2,2,0.083051
3,3,0.135593
